In [ ]:
import time
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, output_notebook, show
from bokeh.io import push_notebook
from bokeh.layouts import column

from pdclient import PdClient
from pdclient.drop import Drop, Dir
from pdclient.heater import get_v4_controller
from pdclient.reservoir import register_driver, ReservoirDriver

PDHOST = 'http://192.168.0.2:7000/rpc'

In [ ]:
# Override dispense function for reservoir
class ReservoirA(ReservoirDriver):
    def __init__(self, descriptor: dict, client: 'PdClient'):
        super().__init__(descriptor, client)
    
    def dispense(self) -> Drop:
        SEQUENCE = [
            (('B', 'C', 'D', 'E', 'exit'), 2.50),
            (('B', 'C', 'exit'), 2.0),
            (('B', 'exit'), 1.5),
            (('A', 'B', 'exit'), 1.5),
        ]
#         SEQUENCE = [
#             (('A', 'B', 'C', 'D', 'E', 'exit'), 1.0),
#             (('B', 'C', 'D', 'E', 'exit'), 1.0),
#             (('A', 'B', 'C', 'D', 'E', 'exit'), 1.0),
#             (('B', 'C', 'D', 'E', 'exit'), 1.0),
#             (('B', 'C', 'exit'), 0.8),
#             (('A', 'B', 'exit'), 0.7),
#             (('A', 'exit'), 0.7),
#             (('exit',), 0.2),
#         ]

        for step in SEQUENCE:
            pins = [self.pin(name) for name in step[0]]
            delay = step[1]
            self.client.enable_pins(pins)
            time.sleep(delay)
        self.client.enable_pins([])

        exit_pin = self.pin('exit')
        exit_loc = self.client.get_grid_location(exit_pin)
        if exit_loc is None:
            raise ValueError(f"The exit pin ({exit_pin}) for reservoir {self.descriptor['id']} is not in the grid")

        return Drop(exit_loc, (1, 1), self.client)

    def ingest(self):
        raise RuntimeError("Unimplemented")
register_driver('reservoirA', ReservoirA)

In [ ]:
def calc_route(loc_a, loc_b):
    # A very simple function to route from point a to point b, assuming
    # the grid is complete and there are no obstacles
    # Move left/right to x coordinate, then move up/down to y coordinate
    steps = []
    x = loc_a[0]
    y = loc_a[1]
    while x > loc_b[0]:
        steps.append(Dir.LEFT)
        x -= 1
    while x < loc_b[0]:
        steps.append(Dir.RIGHT)
        x += 1
    while y > loc_b[1]:
        steps.append(Dir.UP)
        y -= 1
    while y < loc_b[1]:
        steps.append(Dir.DOWN)
        y += 1
    return steps

client = PdClient(PDHOST)
temp_control = get_v4_controller(client)
master_reservoir = client.get_reservoir(4)
replenish_reservoir = client.get_reservoir(3)
heater_drop = Drop((6, 9), (2, 2), client)

def measure_capacitance(pins):
    bulk_capacitance = client.bulk_capacitance()
    return sum([bulk_capacitance[p] for p in pins])

def move_drop_to_heater(res):
    # Destination is above the heater
    DEST = (6, 7)
    
    new_drop = res.dispense()
    drop_capacitance = measure_capacitance(new_drop.pins())
    print("Dispensed drop capacitance: %f", drop_capacitance)
    if(drop_capacitance < 500):
        raise RuntimeError("No drop detected")
    route = calc_route(new_drop.pos, DEST)
    for mv in route:
        result = new_drop.move(mv)
        if(not result['success']):
            raise RuntimeError(f"Error moving drop: {result}")
        time.sleep(0.5)
    # Finally move the drop down to the heater, and activate all four heater 
    # electrodes to center
    new_drop.move_down()
    time.sleep(0.5)
    heater_drop.activate()
    time.sleep(1.0)
    client.enable_pins([])

def measure_heater_capacitance():
    # Get the sum of capacitance measurement for 4 heater electrodes
    return measure_capacitance(heater_drop.pins())

In [ ]:
move_drop_to_heater(replenish_reservoir)

In [ ]:
# Dispense 4 drops from master mix reservoir to the heater
for _ in range(4):
    move_drop_to_heater(master_reservoir)

In [ ]:
# Create a live plot to display temperature and capacitance
output_notebook()
data_source = ColumnDataSource(data=dict(t=[], target_temp=[], drop_temp=[], capacitance=[]))

fig1 = figure(plot_width=600, plot_height=300, background_fill_color="#fafafa")
fig2 = figure(plot_width=600, plot_height=300, background_fill_color="#fafafa")
fig1.line('t', 'drop_temp', source=data_source, line_width=2, color='green', legend_label='T_drop')
fig1.line('t', 'target_temp', source=data_source, line_width=2, color='black', line_dash=[1, 10], legend_label='target')
fig1.legend.location = "top_left"
fig1.yaxis.axis_label='Deg C'
                               
fig2.line('t', 'capacitance', source=data_source, line_width=2, color='blue', legend_label='heater capacitance')
fig2.yaxis.axis_label='counts'
fig2.xaxis.axis_label='Time (sec)'
fig2.legend.location = "top_left"

plot_handle = show(column(fig1, fig2), notebook_handle=True)

In [ ]:
# Store the capacitance to use a reference level for determining when to replenish
initial_heater_capacitance = measure_heater_capacitance()
print(f"Initial heater capacitance: {initial_heater_capacitance}")

def hold_temp(target, hold_time):
    TIMEOUT = 120.0
    temp_control.set_target(target)
    
    print(f"Setting target={target}C")
    # Wait until we reach the target temp
    start_time = time.monotonic()
    while(abs(temp_control.drop_temperature - target) > 1.0):
        data_source.stream({
            't': [time.monotonic()],
            'drop_temp': [temp_control.drop_temperature],
            'target_temp': [target],
            'capacitance': [measure_heater_capacitance()]
        })
        push_notebook(handle=plot_handle)
        if time.monotonic() - start_time > TIMEOUT:
            print(f"Timedout waiting to reach {target}C")
            break
        time.sleep(1.0)
    print(f"Reached {target}C, holding for {hold_time} seconds.")
    # Wait the specified time
    sleep_time = 0.0
    while(sleep_time < hold_time):
        data_source.stream({
            't': [time.monotonic()],
            'drop_temp': [temp_control.drop_temperature],
            'target_temp': [target],
            'capacitance': [measure_heater_capacitance()]
        })
        push_notebook(handle=plot_handle)
        time.sleep(1.0)
        sleep_time += 1.0
    
def pcr_cycle():
    hold_temp(95, 5)
    temp_control.set_target(60)
    while measure_heater_capacitance() < 0.75 * initial_heater_capacitance:
        move_drop_to_heater(replenish_reservoir)
    hold_temp(60, 30)
    hold_temp(72, 20)
    
temp_control.start()
try:
    while True:
        pcr_cycle()
finally:
    temp_control.stop()

In [ ]:
import json
with open('PCRrun-2020-06-17', 'w') as f:
    f.write(json.dumps(data_source.data))